In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 08:00:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [5]:
df_yellow \
    .repartition(4) \
    .write.parquet('data/pq/yellow/2024/10', mode='overwrite')

In [6]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [23]:
df_yellow = spark.read.parquet('data/pq/yellow/2024/*')
df_yellow.registerTempTable('yellow')

In [46]:
df_yellow_count = spark.sql("""
SELECT 
    COUNT(1)
FROM
    yellow
WHERE  tpep_pickup_datetime BETWEEN '2024-10-15 00:00:00' AND '2024-10-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [47]:
df_yellow_long = spark.sql("""
SELECT 
    MAX(timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)) AS longest_trip
FROM yellow
""").show()

+------------+
|longest_trip|
+------------+
|         162|
+------------+



In [49]:
df_zone = spark.read.parquet('data/pq/zone/*')
df_zone.registerTempTable('zone')

In [50]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [57]:
least_frequent_zone = spark.sql("""
SELECT t2.Zone, count(1) as least 
FROM yellow t1 INNER JOIN zone t2 ON t1.PULocationID = t2.LocationID
GROUP BY t2.Zone
ORDER BY least 
LIMIT 1
""").show()

+--------------------+-----+
|                Zone|least|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+

